In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import optuna
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
from utils import calculate_metric

In [2]:
# Set up paths and load data
DATA_FOLDER = "../data"
TRAIN_FEATURES = DATA_FOLDER + '/train_features2.xlsx'
TRAIN_LABELS = DATA_FOLDER + "/train_labels2.xlsx"
TEST_FEATURES = DATA_FOLDER + "/test_features2.xlsx"
TEST_LABELS = DATA_FOLDER + "/test_labels2.xlsx"
OUTPUT_PATH = '../output/xgboost_pca'

# Create output directory if it doesn't exist
output_dir = Path(OUTPUT_PATH)
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
# Load data
train_features = pd.read_excel(TRAIN_FEATURES)
train_labels = pd.read_excel(TRAIN_LABELS)
test_features = pd.read_excel(TEST_FEATURES)
test_labels = pd.read_excel(TEST_LABELS)

print("Training features shape:", train_features.shape)
print("Test features shape:", test_features.shape)

Training features shape: (1293, 317)
Test features shape: (432, 317)


In [4]:
def create_pca_pipeline(n_components=100):
    """
    Create a pipeline with StandardScaler and PCA
    n_components: float between 0 and 1 (explained variance ratio)
    """
    return Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n_components))
    ])

# Separate numerical and categorical features
numerical_features = train_features.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train_features.select_dtypes(include=['object', 'bool']).columns

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', create_pca_pipeline(), numerical_features),
        ('cat', 'passthrough', categorical_features)
    ])

In [9]:
def objective(trial):
    param = {
        # "verbosity": 0,
        "objective": "reg:squarederror",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, log=True),
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        # "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        # "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }
    
    if param["booster"] in "gbtree":
        # sampling according to each tree.
        param["colsample_bytree"] = trial.suggest_float("colsample_bytree", 0.2, 1.0)
        # sampling ratio for training data.
        param["subsample"] = trial.suggest_float("subsample", 0.2, 1.0)
        # use exact for small dataset.
        param["tree_method"] = trial.suggest_categorical("tree_method", ["exact", "auto"])

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 2, 28, step=2)
        # minimum child weight, larger the term more conservative the tree.
        # param["min_child_weight"] = trial.suggest_int("min_child_weight", 1, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        # param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        # param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    
    model = xgb.XGBRegressor(**param, random_state=42)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    score = -cross_val_score(
        pipeline, train_features, train_labels.values.ravel(),
        cv=5, scoring='neg_root_mean_squared_error'
    ).mean()
    
    return score

In [10]:
# Run hyperparameter optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Get best parameters
best_params = study.best_params
print("Best parameters:", best_params)

[I 2025-04-14 11:02:57,525] A new study created in memory with name: no-name-24aac3eb-9d71-4b3d-bdf9-fcfbc97687fe
[I 2025-04-14 11:04:14,982] Trial 0 finished with value: 0.27348939053418764 and parameters: {'n_estimators': 189, 'booster': 'gbtree', 'colsample_bytree': 0.5951706805093191, 'subsample': 0.5171638635986554, 'tree_method': 'auto', 'max_depth': 2, 'eta': 0.021677723770513198}. Best is trial 0 with value: 0.27348939053418764.
[I 2025-04-14 11:05:42,277] Trial 1 finished with value: 0.3334537011308575 and parameters: {'n_estimators': 137, 'booster': 'gblinear'}. Best is trial 0 with value: 0.27348939053418764.
[I 2025-04-14 11:07:06,260] Trial 2 finished with value: 0.33804441293642445 and parameters: {'n_estimators': 104, 'booster': 'gblinear'}. Best is trial 0 with value: 0.27348939053418764.
[I 2025-04-14 11:09:41,528] Trial 3 finished with value: 0.32778398612228815 and parameters: {'n_estimators': 113, 'booster': 'dart', 'max_depth': 8, 'eta': 9.183532752281618e-05, 'sam

Best parameters: {'n_estimators': 293, 'booster': 'gbtree', 'colsample_bytree': 0.5343004529208513, 'subsample': 0.6450299620605517, 'tree_method': 'exact', 'max_depth': 12, 'eta': 0.014062459219759479}


In [5]:
best_params = {'n_estimators': 353,
 'booster': 'dart',
 'max_depth': 6,
 'eta': 0.046653323948485266,
 'sample_type': 'uniform',
 'normalize_type': 'forest',
 'rate_drop': 4.201145869116434e-06,
 'skip_drop': 0.0013199458257448385}

In [6]:
# Train final model with best parameters
final_model = xgb.XGBRegressor(**best_params, random_state=42)
final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', final_model)
])

# Fit the model
final_pipeline.fit(train_features, train_labels.values.ravel())

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=100))]),
                                                  Index(['coupon rate', 'SP500 MD', 'Average daily 1-year SP500 return',
       'Ratio to MA', 'US Corporate Bond Yield Spread',
       'US Corporate Bond Yield Spread(3-5 year)',
       'US Corporate Bond Yield Spread(5-7 year)',
       'US Corporate Bond Yield Spr...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=353, n_jobs=None,
                              normalize_type='forest', ...))])

In [7]:
# Make predictions
train_predictions = final_pipeline.predict(train_features)
test_predictions = final_pipeline.predict(test_features)

# Calculate and print metrics
train_metrics = calculate_metric(train_predictions, train_labels.values.ravel())
test_metrics = calculate_metric(test_predictions, test_labels.values.ravel())

print("\nTraining Metrics:")
print(f"MAE: {train_metrics[0]:.4f}")
print(f"MAPE: {train_metrics[1]:.4f}%")
print(f"RMSE: {train_metrics[2]:.4f}")
print(f"R2: {train_metrics[3]:.4f}")

print("\nTest Metrics:")
print(f"MAE: {test_metrics[0]:.4f}")
print(f"MAPE: {test_metrics[1]:.4f}%")
print(f"RMSE: {test_metrics[2]:.4f}")
print(f"R2: {test_metrics[3]:.4f}")


Training Metrics:
MAE: 0.0203
MAPE: 202.7964%
RMSE: 0.0405
R2: 0.9849

Test Metrics:
MAE: 0.1345
MAPE: 275.3021%
RMSE: 0.2194
R2: 0.5797


In [ ]:
def plot_feature_importance(model, preprocessor):
    # Get feature names after PCA
    n_components = preprocessor.named_transformers_['num'].named_steps['pca'].n_components_
    pca_features = [f'PC{i+1}' for i in range(n_components)]
    cat_features = list(categorical_features)
    
    # Get feature importance
    importance = model.feature_importances_
    
    # Create DataFrame
    feature_importance = pd.DataFrame({
        'Feature': pca_features + cat_features,
        'Importance': importance
    })
    
    # Sort by importance
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    
    # Plot
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(20), x='Importance', y='Feature')
    plt.title('Top 20 Feature Importance')
    plt.tight_layout()
    plt.show()

# Plot feature importance
plot_feature_importance(final_model, preprocessor)

In [ ]:
# Save predictions
predictions_df = pd.DataFrame({
    'Actual': test_labels.values.ravel(),
    'Predicted': test_predictions
})
predictions_df.to_excel(f"{OUTPUT_PATH}/predictions.xlsx", index=False)

# Save model
joblib.dump(final_pipeline, f"{OUTPUT_PATH}/xgboost_pca_model.pkl")